In [24]:
import torch
from torch import nn
from d2l import torch as d2l
import math
batch_size, num_steps = 32, 35
train_iter, vocab = d2l.load_data_time_machine(batch_size, num_steps)

In [108]:
def train_epoch_ch8(net, train_iter, loss, updater, device, use_random_iter):
    """Train a net within one epoch (defined in Chapter 8)."""
    state, timer = None, d2l.Timer()
    metric = d2l.Accumulator(2)  # Sum of training loss, no. of tokens
    for X, Y in train_iter:
        if state is None or use_random_iter:
            # Initialize `state` when either it is the first iteration or
            # using random sampling
            state = net.begin_state(batch_size=X.shape[0], device=device)

        else:
            if isinstance(net, nn.Module) and not isinstance(state, tuple):
                # `state` is a tensor for `nn.GRU`
                state.detach_()
            else:
                # `state` is a tuple of tensors for `nn.LSTM` and
                # for our custom scratch implementation
                for s in state:
                    s.detach_()
        print(X.requires_grad)
        y = Y.T.reshape(-1)
        X, y = X.to(device), y.to(device)
        y_hat, state = net(X, state)
        l = loss(y_hat, y.long()).mean()
        if isinstance(updater, torch.optim.Optimizer):
            updater.zero_grad()
            l.backward()
            grad_clipping(net, 1)
            updater.step()
        else:
            l.backward()
            grad_clipping(net, 1)
            # Since the `mean` function has been invoked
            updater(batch_size=1)
        metric.add(l * d2l.size(y), d2l.size(y))
    return math.exp(metric[0] / metric[1]), metric[1] / timer.stop()

In [12]:
vocab_size, num_hiddens, device = len(vocab), 256, d2l.try_gpu()

In [15]:
num_inputs = vocab_size
lstm_layer = nn.LSTM(num_inputs, num_hiddens)
net = d2l.RNNModel(lstm_layer, len(vocab))

In [20]:
num_epochs, lr = 500, 1
loss = nn.CrossEntropyLoss()
updater = torch.optim.SGD(net.parameters(), lr)

In [22]:
def grad_clipping(net, theta):
    """Clip the gradient."""
    if isinstance(net, nn.Module):
        params = [p for p in net.parameters() if p.requires_grad]
    else:
        params = net.params
    norm = torch.sqrt(sum(torch.sum((p.grad**2)) for p in params))
    if norm > theta:
        for param in params:
            param.grad[:] *= theta / norm

In [109]:
ppl, speed = train_epoch_ch8(net, train_iter, loss, updater, device, False)

False
False
False
False
False
False
False
False


In [103]:
def predict_ch8(prefix, num_preds, net, vocab, device):
    """Generate new characters following the `prefix`."""
    state = net.begin_state(batch_size=1, device=device)
    outputs = [vocab[prefix[0]]]
    get_input = lambda: d2l.reshape(d2l.tensor([outputs[-1]], device=device),
                                    (1, 1))
    for y in prefix[1:]:  # Warm-up period
        _, state = net(get_input(), state)
        outputs.append(vocab[y])
    for _ in range(num_preds):  # Predict `num_preds` steps
        y, state = net(get_input(), state)
        outputs.append(int(y.argmax(dim=1).reshape(1)))
    print(state[0].size())
    print(get_input().size())
    return ''.join([vocab.idx_to_token[i] for i in outputs])

In [78]:
a = ()

In [72]:
prefix = 'time traveller'

In [73]:
vocab[prefix[0]]

3

In [101]:
predict = lambda prefix: predict_ch8(prefix, 50, net, vocab, device)

In [104]:
print(predict('time traveller'))

torch.Size([1, 1, 256])
torch.Size([1, 1])
time traveller te te te te te te te te te te te te te te te te t
